# Looking into DB
https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks?select=books.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('goodreads_data.csv')

In [ ]:
#data.info()
#data.shape

In [4]:
#data.loc[data['Unnamed: 12'].notnull() | data['Unnamed: 13' ].notnull() ].sum()
data.isna().sum()
#it seems that unnamed columns could be eliminated but maybe they have usuful information, for now we are going to keep them

bookID                   0
title                    0
authors                  1
average_rating           1
isbn                     1
isbn13                   1
language_code            1
num_pages                1
ratings_count            1
text_reviews_count       1
publication_date         1
publisher                1
Unnamed: 12           8427
Unnamed: 13           8465
Unnamed: 14           8471
dtype: int64

In [82]:
#Droping rows with extra data
data = data[data['Unnamed: 12'].isna()] 
data = data[data['Unnamed: 13'].isna()]
data = data[data['Unnamed: 14'].isna()]
#droping empty columns
data.drop(['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'language_code', 'bookID'], axis = 1, inplace = True)
print(data.shape)
data.to_csv('books_2.csv', index = False)
#8427 records after droping records with extra columns
#data.info()

(8427, 10)


In [83]:
data =data.dropna()
data.shape
data.isna().sum()

title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
num_pages             0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [84]:
data.reset_index()
data.head(1)
#(8427, 10)

,title,authors,average_rating,isbn,isbn13,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9780439785969,652,2095690,27591,9/16/2006,Scholastic Inc.


In [85]:
data['book_id'] = data.index
data.columns

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'book_id'],
      dtype='object')

# DB Tables to construct
1. ratings (principal table): isbn, average_rating, ratings_count, text_reviews_count
2. books: isbn, title, num_pages, publication_date, book_id    
3. authors: author_id, name    
4. publisher: publisher_id, publisher
5. connector books-authors: book_id, author_id
6. connector books-publishers: book_id, publisher_id 

## ratings

In [75]:
temp = data.copy(deep = True)            # droping null values
#(8427, 10)
temp.columns

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'book_id'],
      dtype='object')

In [78]:
ratings = temp[['average_rating', 'isbn', 'ratings_count', 'text_reviews_count']]

In [79]:
ratings=ratings[['isbn', 'average_rating', 'ratings_count', 'text_reviews_count']]
ratings.head(1)

,isbn,average_rating,ratings_count,text_reviews_count
0,439785960,4.57,2095690,27591


In [80]:
ratings.to_csv('ratings.csv', index =False)

## Books SF

### Books

In [90]:
temp = data.copy(deep = True)            # droping null values
#(8427, 10)
temp.columns

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'book_id'],
      dtype='object')

In [91]:
book = temp[['book_id', 'title', 'isbn', 'num_pages', 'publication_date']]

In [92]:
#changing date format mm/dd/yyyy to yyyy-mm-dd
date_split = pd.concat([book['isbn'], book['publication_date'].str.split('/', expand=True)], axis=1)
book['publication_date'] = date_split[[2, 0, 1]].agg('-'.join, axis=1)
date.head(3)

c:\users\crgbl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    2006-9-16
1     2004-9-1
2    2003-11-1
dtype: object

In [93]:
book = book[['isbn', 'title', 'num_pages', 'publication_date', 'book_id']]
book.to_csv('books.csv', index =False)

## authors

In [56]:
temp = data.copy(deep = True)            # droping null values
#(8427, 10)
temp.columns

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'book_id'],
      dtype='object')

In [57]:
#Creating csv for table authors
temp[['book_id','authors']].head(2)

,book_id,authors
0,0,J.K. Rowling/Mary GrandPré
1,1,J.K. Rowling/Mary GrandPré


In [58]:
authors_split = pd.concat([temp['book_id'], temp['authors'].str.split('/', expand=True)], axis=1)
#Since a book could have more than one author, then in the book table we need to create more than one row fro a book to include
#all authors and fulfill normalization 
#authors_split.dropna(inplace =True)

In [59]:
#Fetching unique values
temp_1 = authors_split.drop('book_id', axis=1)
authors=pd.DataFrame(pd.unique(temp_1.values.ravel()))
print(authors.shape)

authors.dropna(inplace =True)         
print(authors.shape)

(7320, 1)
(7319, 1)


In [61]:
authors['author_id'] = authors.index
authors.rename(columns ={0: 'name'}, inplace =True)
authors = authors[['author_id', 'name']]
authors.head()
authors['name']= authors['name'].str.replace("'"," ")
#author_id moust be a primary key

In [62]:
authors.to_csv('authors.csv', index = False)

## connector table book-author

In [22]:
connec = pd.DataFrame()
for i in range(authors_split.shape[1]-1):
    tem = authors_split[['book_id', i]].dropna()
    tem.rename(columns = {i: 'name'}, inplace =True)
    connec = connec.append(tem)
connec.sort_values('book_id', inplace=True)
print(connec)
#authors_split

      book_id               name
0           0       J.K. Rowling
0           0      Mary GrandPré
1           1       J.K. Rowling
1           1      Mary GrandPré
2           2       J.K. Rowling
...       ...                ...
8468     8468   Anthony Bourdain
8469     8469           D.T. Max
8470     8470  Leonard S. Marcus
8470     8470   Ursula Nordstrom
8470     8470     Maurice Sendak

[14418 rows x 2 columns]


In [24]:
connect = connec.copy(deep=True)

In [25]:
authors.head(3) 

,author_id,name
0,0,J.K. Rowling
1,1,Mary GrandPré
3,3,W. Frederick Zimmerman


In [26]:
#connecting table book_id and author_id
connect =connect.set_index('name').join(authors.set_index('name'))
connect.sort_values('book_id', inplace=True)
connect.to_csv('connector_book_author_ids.csv', index = False)

## publishers

In [68]:
temp = data.copy(deep = True)            # droping null values
#(8427, 10)
temp.columns

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'book_id'],
      dtype='object')

In [69]:
#Creating csv for table authors
temp[['book_id','publisher']].head(2)

,book_id,publisher
0,0,Scholastic Inc.
1,1,Scholastic Inc.


In [70]:
publishers_split = pd.concat([temp['book_id'], temp['publisher'].str.split('/', expand=True)], axis=1)
#Since several books could be published by the same publisher, then we need to create a table for them. Also, it is necessary 
# to create a connector table to reduced memory and improve clarity 
#8427 records

In [71]:
#Fetching unique values
temp_1 = publishers_split.drop('book_id', axis=1)
publishers=pd.DataFrame(pd.unique(temp_1.values.ravel()))
print(publishers.shape)

publishers.dropna(inplace =True)         
print(publishers.shape)

(1941, 1)
(1940, 1)


In [73]:
publishers['publisher_id'] = publishers.index
publishers.rename(columns ={0: 'publisher'}, inplace =True)
publishers = publishers[['publisher_id', 'publisher']]
publishers.head()
publishers['publisher']= publishers['publisher'].str.replace("'"," ")
#author_id moust be a primary key

In [74]:
publishers.to_csv('publishers.csv', index = False)

## Connector table book-publishers

In [33]:
connec = pd.DataFrame()
tem = publishers_split[['book_id', 0]].dropna()
tem.rename(columns = {0: 'publisher'}, inplace =True)
connec = connec.append(tem)
connec.sort_values('book_id', inplace=True)
print(connec)
#authors_split

      book_id                      publisher
0           0                Scholastic Inc.
1           1                Scholastic Inc.
2           2                     Scholastic
3           3                Scholastic Inc.
4           4                     Scholastic
...       ...                            ...
8465     8465                        Taschen
8467     8467                Canongate Books
8468     8468                Canongate Books
8469     8469  Random House Trade Paperbacks
8470     8470                  HarperCollins

[8426 rows x 2 columns]


In [34]:
publishers.head(3)
#connecting table book_id and publisher_id
connec =connec.set_index('publisher').join(publishers.set_index('publisher'))
connec.sort_values('book_id', inplace=True)
connec.to_csv('connector_book_publisher_ids.csv', index = False)

Looking if there are some repeated records

In [94]:
book_id_publishers = set(connec['book_id'].tolist())

In [95]:
book_id_books = set(book['book_id'].tolist())

In [96]:
book_id_publishers-book_id_books

set()

### Identifiers

In [ ]:
temp = data[data['isbn'].notnull()]              # droping null values
(temp.duplicated(subset=['isbn']) == True).sum() #looking for duplicates

In [ ]:
identifiers = pd.DataFrame()
identifiers['isbn'] =temp['isbn']
identifiers['isbn13'] =temp['isbn13']
#isbn must be the primary key
identifiers.head(3)

In [ ]:
identifiers.to_csv('identifier.csv', index = False)